In [1]:
import os
import sys
sys.path.append('../')

from bimpm.graph import Graph
import tensorflow as tf
from bimpm import args
from utils.load_data import load_char_word_static_data

/home/jerry/workshop/virtualenv/alpha-mind/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jerry/workshop/virtualenv/alpha-mind/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jerry/workshop/virtualenv/alpha-mind/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jerry/workshop/v

In [2]:
p_index, h_index, p_vec, h_vec, label = load_char_word_static_data('../input/train.csv', data_size=None)
p_index_dev, h_index_dev, p_vec_dev, h_vec_dev, label_dev = load_char_word_static_data('../input/dev.csv', data_size=100)

Building prefix dict from the default dictionary ...
I1203 21:29:13.071447 139953856177920 __init__.py:111] Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
I1203 21:29:13.090180 139953856177920 __init__.py:131] Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
I1203 21:29:13.621901 139953856177920 __init__.py:145] Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/home/jerry/workshop/virtualenv/alpha-mind/lib/python3.6/site-packages/jieba/__init__.py", line 152, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmpxm0o3xqr' -> '/tmp/jieba.cache'
E1203 21:29:13.663039 139953856177920 __init__.py:154] Dump cache file failed.
Traceback (most recent call last):
  File "/home/jerry/workshop/virtualenv/alpha-mind/lib/python3.6/site-packages/jieba/__init__.py", line 152, in initialize
    _r

In [3]:
import numpy as np
np.shape(p_index), np.shape(h_index), np.shape(p_vec), np.shape(h_vec), np.shape(label)

((100000, 15), (100000, 15), (100000, 15, 100), (100000, 15, 100), (100000,))

In [4]:
p_index_holder = tf.placeholder(name='p_index', shape=(None, args.max_char_len), dtype=tf.int32)
h_index_holder = tf.placeholder(name='h_index', shape=(None, args.max_char_len), dtype=tf.int32)
p_vec_holder = tf.placeholder(name='p_vec', shape=(None, args.max_word_len, args.word_embedding_len),
                              dtype=tf.float32)
h_vec_holder = tf.placeholder(name='h_vec', shape=(None, args.max_word_len, args.word_embedding_len),
                              dtype=tf.float32)
label_holder = tf.placeholder(name='label', shape=(None,), dtype=tf.int32)



In [5]:
dataset = tf.data.Dataset.from_tensor_slices((p_index_holder, h_index_holder, p_vec_holder, h_vec_holder, label_holder))
dataset = dataset.batch(args.batch_size).repeat(args.epochs)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
model = Graph()
saver = tf.train.Saver()

W1203 21:29:36.548731 139953856177920 deprecation.py:323] From <ipython-input-5-afc66e196d52>:3: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
W1203 21:29:36.622682 139953856177920 deprecation_wrapper.py:119] From ../bimpm/graph.py:10: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1203 21:29:36.635999 139953856177920 deprecation_wrapper.py:119] From ../bimpm/graph.py:19: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W1203 21:29:36.636574 139953856177920 deprecation.py:506] From /home/jerry/workshop/virtualenv/alpha-mind/lib/python3.6/site-packages/tensorflow/pytho

In [6]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.7

In [7]:
with tf.Session(config=config)as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(iterator.initializer, feed_dict={p_index_holder: p_index,
                                              h_index_holder: h_index,
                                              p_vec_holder: p_vec,
                                              h_vec_holder: h_vec,
                                              label_holder: label})
    steps = int(len(label) / args.batch_size)
    for epoch in range(1):
        for step in range(steps):
            try:
                p_index_batch, h_index_batch, p_vec_batch, h_vec_batch, label_batch = sess.run(next_element)
                loss, _, predict, acc = sess.run(
                    [model.loss, model.train_op, model.predict, model.accuracy],
                    feed_dict={model.p: p_index_batch,
                               model.h: h_index_batch,
                               model.p_vec: p_vec_batch,
                               model.h_vec: h_vec_batch,
                               model.y: label_batch,
                               model.keep_prob: args.keep_prob})
                print('epoch:', epoch, ' step:', step, ' loss:', loss / args.batch_size, ' acc:', acc)
            except tf.errors.OutOfRangeError:
                print('\n')

        predict, acc = sess.run([model.predict, model.accuracy],
                                feed_dict={model.p: p_index_dev,
                                           model.h: h_index_dev,
                                           model.p_vec: p_vec_dev,
                                           model.h_vec: h_vec_dev,
                                           model.y: label_dev,
                                           model.keep_prob: 1})
        print('epoch:', epoch, ' dev acc:', acc)
        saver.save(sess, f'../output/diin/diin_{epoch}.ckpt')
        print('save model done')
        print('\n')

ResourceExhaustedError: OOM when allocating tensor of shape [10000,512] and type float
	 [[node dense_1/kernel/Adam_1/Initializer/zeros (defined at ../bimpm/graph.py:184) ]]

Original stack trace for 'dense_1/kernel/Adam_1/Initializer/zeros':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jerry/workshop/virtualenv/alpha-mind/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jerry/workshop/virtualenv/alpha-mind/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jerry/workshop/virtualenv/alpha-mind/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/jerry/workshop/virtualenv/alpha-mind/lib/python3.6/site-packages/tornado/platform/asyncio.py", lin